In [1]:
try:

    import tensorflow as tf
    import cv2
    import os
    import pickle
    import numpy as np
    print("Library Loaded Successfully ..........")
except:
    print("Library not Found ! ")


class MasterImage(object):

    def __init__(self,PATH='', IMAGE_SIZE = 50):
        self.PATH = PATH
        self.IMAGE_SIZE = IMAGE_SIZE

        self.image_data = []
        self.x_data = []
        self.y_data = []
        self.CATEGORIES = []

        # This will get List of categories
        self.list_categories = []

    def get_categories(self):
        for path in os.listdir(self.PATH):
            if '.DS_Store' in path:
                pass
            else:
                self.list_categories.append(path)
        print("Found Categories ",self.list_categories,'\n')
        return self.list_categories

    def Process_Image(self):
        try:
            """
            Return Numpy array of image
            :return: X_Data, Y_Data
            """
            self.CATEGORIES = self.get_categories()
            for categories in self.CATEGORIES:                                                  # Iterate over categories

                train_folder_path = os.path.join(self.PATH, categories)                         # Folder Path
                class_index = self.CATEGORIES.index(categories)                                 # this will get index for classification

                for img in os.listdir(train_folder_path):                                       # This will iterate in the Folder
                    new_path = os.path.join(train_folder_path, img)                             # image Path

                    try:        # if any image is corrupted
                        image_data_temp = cv2.imread(new_path,cv2.IMREAD_GRAYSCALE)                 # Read Image as numbers
                        image_temp_resize = cv2.resize(image_data_temp,(self.IMAGE_SIZE,self.IMAGE_SIZE))
                        self.image_data.append([image_temp_resize,class_index])
                    except:
                        pass

            data = np.asanyarray(self.image_data)

            # Iterate over the Data
            for x in data:
                self.x_data.append(x[0])        # Get the X_Data
                self.y_data.append(x[1])        # get the label

            X_Data = np.asarray(self.x_data) / (255.0)      # Normalize Data
            Y_Data = np.asarray(self.y_data) 

            # reshape x_Data

            X_Data = X_Data.reshape(-1, self.IMAGE_SIZE, self.IMAGE_SIZE, 1)

            return X_Data, Y_Data
        except:
            print("Failed to run Function Process Image ")

    def pickle_image(self):

        """
        :return: None Creates a Pickle Object of DataSet
        """
        # Call the Function and Get the Data
        X_Data,Y_Data = self.Process_Image()

        # Write the Entire Data into a Pickle File
        pickle_out = open('X_Data','wb')
        pickle.dump(X_Data, pickle_out)
        pickle_out.close()

        # Write the Y Label Data
        pickle_out = open('Y_Data', 'wb')
        pickle.dump(Y_Data, pickle_out)
        pickle_out.close()

        print("Pickled Image Successfully ")
        return X_Data,Y_Data

    def load_dataset(self):

        try:
            # Read the Data from Pickle Object
            X_Temp = open('X_Data','rb')
            X_Data = pickle.load(X_Temp)

            Y_Temp = open('Y_Data','rb')
            Y_Data = pickle.load(Y_Temp)

            print('Reading Dataset from PIckle Object')

            return X_Data,Y_Data

        except:
            print('Could not Found Pickle File ')
            print('Loading File and Dataset  ..........')

            X_Data,Y_Data = self.pickle_image()
            return X_Data,Y_Data


if __name__ == "__main__":
    path = '/home/jesterrexx/Documents/Dataset/RPS/Rock-Paper-Scissors/train'
    a = MasterImage(PATH=path,
                    IMAGE_SIZE=70)

    X_Data,Y_Data = a.load_dataset()
    print(X_Data.shape)

Library Loaded Successfully ..........
Reading Dataset from PIckle Object
(8005, 80, 80, 1)


In [2]:
print('X_Data: ',X_Data.shape)
print('Y_Data: ',Y_Data.shape)

X_Data:  (8005, 80, 80, 1)
Y_Data:  (8005,)


In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_Data, Y_Data, test_size=0.15, random_state=42)
number_of_train = x_train.shape[1]
number_of_test = x_test.shape[1]

In [4]:
m_train = y_train.shape[0]
m_test = y_test.shape[0]
num_px = x_train.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(x_train.shape))
print ("train_set_y shape: " + str(y_train.shape))
print ("test_set_x shape: " + str(x_test.shape))
print ("test_set_y shape: " + str(y_test.shape))

Number of training examples: m_train = 6804
Number of testing examples: m_test = 1201
Height/Width of each image: num_px = 80
Each image is of size: (80, 80, 3)
train_set_x shape: (6804, 80, 80, 1)
train_set_y shape: (6804,)
test_set_x shape: (1201, 80, 80, 1)
test_set_y shape: (1201,)


In [5]:

train_set_x_flatten = x_train.reshape(x_train.shape[0],-1).T                                            
test_set_x_flatten = x_test.reshape(x_test.shape[0],-1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(y_train.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(y_test.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

train_set_x_flatten shape: (6400, 6804)
train_set_y shape: (6804,)
test_set_x_flatten shape: (6400, 1201)
test_set_y shape: (1201,)
sanity check after reshaping: [0.99607843 0.99607843 0.99607843 0.99607843 0.99607843]


In [6]:
train_set_x = train_set_x_flatten / 255.
test_set_x = test_set_x_flatten / 255.

In [7]:
def tanh(z):
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))


In [8]:
print(tanh(9.5))

0.9999999887944071


In [9]:
def leakyrelu(z, alpha):
	return max(alpha * z, z)

In [10]:
print ("leakyrelu(0) = " + str(leakyrelu(0,0.2)))
print ("leakyrelu(9.2) = " + str(leakyrelu(9.2,0.2)))

leakyrelu(0) = 0.0
leakyrelu(9.2) = 9.2


In [11]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

In [12]:
# GRADED FUNCTION: initialize_with_zeros
import random
def initialize_with_zeros(dim):
    m=5
    #w = np.zeros(shape=(dim, 1))
    w=np.random.rand(dim, 1)
    b = 2
    
    return w, b

In [13]:
train_set_x[0]

array([0.00390619, 0.00150711, 0.0030296 , ..., 0.00036909, 0.00143022,
       0.00252211])

In [14]:
dim =  train_set_x.shape[0]
w,b = initialize_with_zeros(dim)

In [15]:
print('Values of weight: ',w)
print('Values of bias: ',b)

Values of weight:  [[0.12237231]
 [0.37467684]
 [0.05917774]
 ...
 [0.74955912]
 [0.7184552 ]
 [0.80805638]]
Values of bias:  2


# Forward and Backward Propagation

In [16]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    A = tanh(np.dot(w.T, X) + b)  # compute activation
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A)))  # compute cost
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = (1 / m) * np.dot(X, (A - Y).T)
    db = (1 / m) * np.sum(A - Y)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [17]:
w, b, X, Y = np.random.rand(2,1)*0.01, 0 ,np.array([[1,2], [3,4]]), np.array([[1, 0]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[-0.46755892]
 [-1.42735816]]
db = -0.47989962316801965
cost = 2.095329717489322


# Optimization

In [18]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):

    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, Y)

        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule 
        w = w - learning_rate * dw  # need to broadcast
        b = b - learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [19]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.01, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print(cost)

w = [[-0.09116913]
 [ 0.13322889]]
b = 0.11360456008127823
dw = [[ 0.13295075]
 [-0.03497545]]
db = -0.08396309832897758
2.095329717489322


# Predict

In [20]:
# GRADED FUNCTION: predict

def predict(w, b, X):

    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = tanh(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        # Convert probabilities a[0,i] to actual predictions p[0,i]
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [21]:
print("predictions = " + str(predict(w, b, X)))

predictions = [[0. 0.]]


In [22]:
# GRADED FUNCTION: model

def model(train_set_x, Y_train, test_set_x, Y_test, num_iterations=2000, learning_rate='', print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    ### START CODE HERE ###
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(train_set_x.shape[0])

    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize(w, b, train_set_x ,y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w, b, test_set_x)
    Y_prediction_train = predict(w, b, train_set_x)

    ### END CODE HERE ###

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [23]:
d = model(train_set_x, y_train, test_set_x, y_test, num_iterations = 2000, learning_rate = 0.0009, print_cost = True)

Cost after iteration 0: 7.285701
Cost after iteration 100: 7.240587
Cost after iteration 200: 7.195474
Cost after iteration 300: 7.150361
Cost after iteration 400: 7.105248
Cost after iteration 500: 7.060135
Cost after iteration 600: 7.015022
Cost after iteration 700: 6.969910
Cost after iteration 800: 6.924798
Cost after iteration 900: 6.879686
Cost after iteration 1000: 6.834574
Cost after iteration 1100: 6.789463
Cost after iteration 1200: 6.744352
Cost after iteration 1300: 6.699242
Cost after iteration 1400: 6.654132
Cost after iteration 1500: 6.609022
Cost after iteration 1600: 6.563914
Cost after iteration 1700: 6.518805
Cost after iteration 1800: 6.473697
Cost after iteration 1900: 6.428590
train accuracy: 50.44091710758378 %
test accuracy: 47.710241465445456 %
